# Orders Forecasting Challenge

## References

## Import Python Libraries

In [579]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from scipy.stats import boxcox, boxcox_normplot
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif, RFE, SelectFromModel
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score, r2_score, mean_squared_error, mean_absolute_error, median_absolute_error
import requests
import datetime as dt

# Import Notebook Functions

In [580]:
# get regression metric functions from stored python file
filepath = "https://raw.githubusercontent.com/notfakearcher/julian/main/01_practice/machine_learning/supervised_learning/regression/regression_metrics.py"
request = requests.get(filepath)
with open("regression_metrics.py", "wb") as f:
  f.write(request.content)
from regression_metrics import jra_regression_metrics

## Global Variables

In [581]:
# random state seed for repeatability
random_state = 4781

# colors
c1 = 'grey'
c2 = 'red'
c3 = 'blue'

# root folder
root_folder = 'https://raw.githubusercontent.com/notfakearcher/julian/main/04_projects/orders_forecasting/'

# data folder
data_folder = root_folder + 'data/'

## Import Dataset

In [582]:
# Source: 
# 1. ....

# Dataset Column Overview: 
# ------------------------------------------------------------------------------

# X01: 
# X02: 
# X03: 
# X04: 
# X05: 
# X06: 
# X07: 
# X08: 
# X09: 
# X10: 
# X11: 
# X12: 
# X13: 
# X14: 
# X15: 
# X16: 
# X17: 
# X18: 
# X19: 
# X20: 
# X21: 
# X22: 
# X23: 
# X24: 
# X25: 
#   y: 


In [583]:
# load train.csv
filepath1 = data_folder + 'train.csv'
df_train = pd.read_csv(filepath1, header = 0)
df_train.sample(5)

,warehouse,date,orders,holiday_name,holiday,shutdown,mini_shutdown,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,user_activity_1,user_activity_2,id
289,Prague_1,2021-09-21,7570.0,NaN,0,0,0,0,0,0,0,0.0,0,0.00,0.0,1680.0,32013.0,Prague_1_2021-09-21
7168,Budapest_1,2023-09-20,5844.0,NaN,0,0,0,0,0,0,0,0.0,0,0.00,0.0,2967.0,24057.0,Budapest_1_2023-09-20
5852,Frankfurt_1,2023-02-09,1456.0,NaN,0,0,0,0,0,0,0,1.0,0,0.00,0.0,826.0,6366.0,Frankfurt_1_2023-02-09
2121,Brno_1,2023-06-24,6783.0,NaN,0,0,0,0,0,0,0,1.0,0,0.11,0.0,2284.0,30116.0,Brno_1_2023-06-24
4701,Prague_3,2024-01-05,6210.0,NaN,0,0,0,0,0,0,0,0.0,0,0.00,0.0,958.0,23072.0,Prague_3_2024-01-05


In [584]:
# load train_calendar.csv
filepath1 = data_folder + 'train_calendar.csv'
df_train_calendar = pd.read_csv(filepath1, header = 0)

# add id column
df_train_calendar['id'] = df_train_calendar['warehouse'] + "_" + df_train_calendar['date']
df_train_calendar.sample(5)

,date,holiday_name,holiday,shutdown,mini_shutdown,warehouse_limited,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,warehouse,id
8790,2020-08-20,NaN,0,0,0,0,0,0,0,0,0.0,0,0.00,0.0,Munich_1,Munich_1_2020-08-20
8554,2021-12-15,NaN,0,0,0,0,0,0,0,0,0.0,0,0.00,0.0,Munich_1,Munich_1_2021-12-15
4849,2019-08-12,NaN,0,0,0,0,0,0,0,0,0.0,0,5.40,0.0,Prague_2,Prague_2_2019-08-12
13110,2022-02-23,NaN,0,0,0,0,0,0,0,0,0.0,0,2.52,0.0,Budapest_1,Budapest_1_2022-02-23
1237,2021-03-16,NaN,0,0,0,0,0,0,0,0,0.0,0,0.50,0.0,Prague_1,Prague_1_2021-03-16


In [585]:
# left join train and train_calendar
df_all = pd.merge(df_train, df_train_calendar, on = 'id', how = 'left')

# order columns
ordered_cols = np.sort(df_all.columns)
df_all = df_all[ordered_cols]

# replace object columns that are null with ''
object_cols = df_all.select_dtypes(include = 'object').columns.copy()
cond = df_all.loc[:, object_cols].isna()
df_all[cond] = ''

# replace numeric columns that are null with 0
number_cols = df_all.select_dtypes(include = 'number').columns.copy()
cond = df_all.loc[:, number_cols].isna()
df_all[cond] = 0

df_all.sample(5)

,blackout_x,blackout_y,date_x,date_y,frankfurt_shutdown_x,frankfurt_shutdown_y,holiday_name_x,holiday_name_y,holiday_x,holiday_y,...,shutdown_y,snow_x,snow_y,user_activity_1,user_activity_2,warehouse_limited,warehouse_x,warehouse_y,winter_school_holidays_x,winter_school_holidays_y
4247,0,0,2022-10-06,2022-10-06,0,0,,,0,0,...,0,0.0,0.0,955.0,18940.0,0,Prague_3,Prague_3,0,0
2260,0,0,2023-11-10,2023-11-10,0,0,,,0,0,...,0,0.0,0.0,2477.0,35114.0,0,Brno_1,Brno_1,0,0
6397,0,0,2021-07-14,2021-07-14,0,0,,,0,0,...,0,0.0,0.0,2953.0,17901.0,0,Budapest_1,Budapest_1,0,0
3131,0,0,2022-12-22,2022-12-22,0,0,,,0,0,...,0,4.0,4.0,1413.0,24085.0,0,Prague_2,Prague_2,0,0
3654,0,0,2021-02-19,2021-02-19,0,0,,,0,0,...,0,0.0,0.0,997.0,18373.0,0,Prague_3,Prague_3,0,0


In [586]:
# get columns with x in name
x_cols = df_all.filter(like = 'x').columns

for x_col in x_cols:
  # get y_col
  y_col = re.sub(pattern = 'x', repl = 'y', string = x_col)

  # check where x_col and y_col do not match in dataframe
  cond = df_all[x_col] != df_all[y_col]
  
  # replace where does not match with max value
  df_all.loc[cond, x_col] = np.max(df_all[[x_col, y_col]], axis = 1)
  
  # add new column
  new_col = re.sub(pattern = '_x', repl = '', string = x_col)
  df_all[new_col] = df_all[x_col]
  
  # drop existing x_col and y_col
  df_all = df_all.drop([x_col, y_col], errors = 'ignore', axis = 1)

# order columns
ordered_cols = np.sort(df_all.columns)
df_all = df_all[ordered_cols]

df_all.sample(5)

,blackout,date,frankfurt_shutdown,holiday,holiday_name,id,mini_shutdown,mov_change,orders,precipitation,school_holidays,shops_closed,shutdown,snow,user_activity_1,user_activity_2,warehouse,warehouse_limited,winter_school_holidays
3578,0,2024-03-15,0,0,,Prague_2_2024-03-15,0,0.0,7140.0,0.1,0,0,0,0.0,1325.0,25481.0,Prague_2,0,0
5641,1,2022-06-02,0,0,,Frankfurt_1_2022-06-02,0,0.0,1172.0,0.0,0,0,0,0.0,862.0,5060.0,Frankfurt_1,0,0
5647,0,2022-06-10,0,0,,Frankfurt_1_2022-06-10,0,0.0,1387.0,0.0,0,0,0,0.0,923.0,5262.0,Frankfurt_1,0,0
4596,0,2023-09-21,0,0,,Prague_3_2023-09-21,0,0.0,4852.0,0.0,0,0,0,0.0,936.0,20151.0,Prague_3,0,0
253,0,2021-08-16,0,0,,Prague_1_2021-08-16,0,0.0,7134.0,0.6,0,0,0,0.0,1654.0,30430.0,Prague_1,0,0


## Feature Engineering

In [587]:
# convert date text column to date type
df_all['date'] = df_all['date'].astype('datetime64[ns]')
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7340 entries, 0 to 7339
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   blackout                7340 non-null   int64         
 1   date                    7340 non-null   datetime64[ns]
 2   frankfurt_shutdown      7340 non-null   int64         
 3   holiday                 7340 non-null   int64         
 4   holiday_name            7340 non-null   object        
 5   id                      7340 non-null   object        
 6   mini_shutdown           7340 non-null   int64         
 7   mov_change              7340 non-null   float64       
 8   orders                  7340 non-null   float64       
 9   precipitation           7340 non-null   float64       
 10  school_holidays         7340 non-null   int64         
 11  shops_closed            7340 non-null   int64         
 12  shutdown                7340 non-null   int64   

In [588]:
# get column for day
df_all['day'] = df_all['date'].dt.day

# get column for month
df_all['month'] = df_all['date'].dt.month

# get column for year
df_all['year'] = df_all['date'].dt.year

# get column for weekday
df_all['weekday'] = df_all['date'].dt.weekday + 1

# get column for day of year
df_all['day_of_year'] = df_all['date'].dt.dayofyear

# get column for quarter
df_all['quarter'] = df_all['date'].dt.quarter

# get column for is leap year
df_all['is_leap_year'] = df_all['date'].dt.is_leap_year.astype('int')

# order columns
ordered_cols = np.sort(df_all.columns)
df_all = df_all[ordered_cols]

df_all.sample(5)

,blackout,date,day,day_of_year,frankfurt_shutdown,holiday,holiday_name,id,is_leap_year,mini_shutdown,...,shops_closed,shutdown,snow,user_activity_1,user_activity_2,warehouse,warehouse_limited,weekday,winter_school_holidays,year
5245,0,2023-03-07,7,66,0,0,,Munich_1_2023-03-07,0,0,...,0,0,0.0,542.0,16407.0,Munich_1,0,2,0,2023
1251,0,2021-02-02,2,33,0,0,,Brno_1_2021-02-02,0,0,...,0,0,0.0,2330.0,26852.0,Brno_1,0,2,0,2021
6463,0,2021-09-19,19,262,0,0,,Budapest_1_2021-09-19,0,0,...,0,0,0.0,3010.0,18562.0,Budapest_1,0,7,0,2021
571,0,2022-07-01,1,182,0,0,,Prague_1_2022-07-01,0,0,...,0,0,0.0,1633.0,34899.0,Prague_1,0,5,0,2022
712,0,2022-11-19,19,323,0,0,,Prague_1_2022-11-19,0,0,...,0,0,2.0,1646.0,36914.0,Prague_1,0,6,0,2022


## Exploration

In [590]:
# print columns that have only one unique value
for col in df_all.columns:
  if len(df_all[col].unique()) == 1:
    print(col)

warehouse_limited


## Outputs

In [589]:
# # save all data
# filepath1 = data_folder + 'all_train.csv'
# df_all.to_csv(path_or_buf = filepath1, header = True, index = False)